# Generating Subsets for testing


1. Create target state-level subsets for NHGIS base crosswalks
1. Create target state-level subsets for NHGIS base tabular data
1. Record unit tests values for posterity


***This is currently only intended for use with block-level data as base units.***


**James Gaboardi** **(<jgaboardi@gmail.com>), 2020-05**

In [1]:
%load_ext watermark
%watermark

2020-05-22T15:34:21-04:00

CPython 3.7.6
IPython 7.13.0

compiler   : Clang 9.0.1 
system     : Darwin
release    : 19.4.0
machine    : x86_64
processor  : i386
CPU cores  : 4
interpreter: 64bit


In [2]:
import inspect
import nhgisxwalk
import numpy
import pandas

%load_ext autoreload
%autoreload 2
%watermark -w
%watermark -iv

watermark 2.0.2
numpy      1.18.1
nhgisxwalk 0.0.2
pandas     1.0.3



In [3]:
def build_subset(
    stfips,
    src_year,
    src_geog,
    trg_year,
    trg_geog,
    tabular,
    tabular_dir="",
    xwalk_dir="",
    code_type="GJOIN",
    out_dir="../testing_data_subsets",
    ur_path=None,
    aux_tabular=None,
    aux_tabular_dir=None,
    
):
    """Create and write out state-level subsets for NHGIS base crosswalks
    and the associated base tabular data. This is currently only intended
    for use with block-level data.
    
    Note: This function generates subsets of
        (1) the base source state units that
            correspond to the *target* state
            base units (block-block);
        (2) the base source state tabular data
            associated with (1); and,
        (3) [optional] the source state tabular data associated with (1).
            
        Since it is possible for the boundaries of any census geographies
        to change over time, some geographies that were contained in
        State A in Source Year may not be contained in State A in Target
        Year. Therefore, when generating the subset it is necessary to
        fetch *all* possible source units that correspond with target units.
        In order to do this, the containing counties for source base units
        in non-target states are identified and included in the source unit
        subset. Take a subset of Delaware for example; If a 1990-2010
        crosswalk is desired, a small portion of New Jersey must be
        included because the boundary of Delaware in 2010 included an area
        that was New Jersey in 1990.
    
    Parameters
    ----------
    
    stfips : str
        State FIPS code for subset (target year)
        
    src_year : str
        Source year.
    
    src_geog : str
        Source geography.
    
    trg_year : str
        Target year.
    
    trg_geog : str
        Target geography.
    
    tabular : str
        Tabular geography type (as a base for file name).
    
    tabular_dir : str
        Path to the tabular geography file. Default is ''.
    
    xwalk_dir : str
        Path to the crosswalk file. Default is ''.
    
    code_type : str
        Code type/format. Default is 'GJOIN'.
    
    out_dir : str
        Default is '../testing_data_subsets'.
    
    ur_path : str
        Path to Urban/Rural code data for 2000 blocks.
    
    aux_tabular : str
        Auxillary tabular data geography type (as a base for file name).
        Currently used for 1990 BGPs.
    
    Returns
    -------
    
    xwalk : pandas.DataFrame
        Subset of the base crosswalk.
    
    tab : pandas.DataFrame
        Subset of the base tabular data.
    
    """
    
    # 1. read in base_xwalk
    src_col = "%s%s" % (code_type, src_year)
    trg_col = "%s%s" % (code_type, trg_year)
    dtype = {c:str for c in [src_col, trg_col]}
    ct = code_type.lower()[:2]
    xwalk_base_name = "/nhgis_%s%s_%s%s_%s.csv.zip"
    f_xwalk = xwalk_base_name % (src_geog, src_year, trg_geog, trg_year, ct)
    xwalk = pandas.read_csv(xwalk_dir+f_xwalk, index_col=0, dtype=dtype)
    
    # 2. create temp column in base_xwalk for trg_year, stfips
    temp_col = "state%s" % trg_year
    xwalk[temp_col] = xwalk[trg_col].map(lambda x: x[1:3])
    
    # 3. subset base_xwalk, remove temp column, reset index, and write out
    xwalk = xwalk[xwalk[temp_col] == stfips]
    xwalk = xwalk[xwalk.columns[:-1]]
    xwalk.reset_index(inplace=True, drop=True)
    xwalk.to_csv(out_dir+f_xwalk)
    
    # 4. read in base_tabular
    dtype = ["GISJOIN", "YEAR", "STATE", "STATEA", "COUNTY", "COUNTYA"]
    dtype += ["CTY_SUB", "CTY_SUBA", "PLACE", "PLACEA", "TRACTA"]
    dtype += ["BLCK_GRPA", "AIANHHA", "URBRURALA", "NAME"]
    dtype += ["URB_AREAA", "CDA", "RES_TRSTA", "ANRCA"]
    dtype = {c:str for c in dtype}
    f_tab = "%s_%s" % (src_year, tabular)
    f_tab_in = "/%s/%s.csv" % (f_tab, f_tab)
    tab = pandas.read_csv(tabular_dir+f_tab_in, index_col=0, dtype=dtype)
    
    # 5. read in auxillary_tabular
    if src_year == "1990" and aux_tabular:
        f_aux = "%s_%s" % (src_year, aux_tabular)
        f_aux_in = "/%s/%s.csv" % (f_aux, f_aux)
        aux = pandas.read_csv(tabular_dir+f_aux_in, dtype=str)
        
        # create subset of main state
        _sa, _ca = "STATEA", "COUNTYA"
        aux_main = aux[aux[_sa] == stfips].copy()
        
        # slice out counties from non-main states
        src_st, src_ct = "st%s" % src_year, "ct%s" % src_year
        xwalk[src_st] = xwalk[src_col].map(
            lambda x: x if str(x) == "nan" else x[1:3]
        )
        non_main = [stf for stf in xwalk[src_st].unique() if stf != stfips]
        non_main.remove(numpy.nan)
        non_main_sts = xwalk[xwalk[src_st].isin(non_main)].copy()
        non_main_sts[src_ct] = non_main_sts[src_col].map(lambda x: x [4:7])
        
        # overwrite `aux` with the subset of non-main states
        aux = aux[aux[_sa].isin(non_main)]
        for nmst in non_main:
            # unique county codes from source state that are in target state
            nmct = non_main_sts[non_main_sts[src_st] == nmst][src_ct].unique()
            # slice out tablualr data only for needed counties in state
            aux_nonmain = aux[((aux[_sa]==nmst)&(aux[_ca].isin(nmct)))].copy()
            # append to the main auxillary tabular data
            aux_main = aux_main.append(aux_nonmain, ignore_index=True)
        aux = aux_main
        aux.to_csv("%s/%s.csv.zip"%(out_dir,f_aux))
        
    # 6. subset base_tabular by stfips and extract UR code for 2000 blocks
    tab = tab[tab["STATEA"] == stfips]
    if src_year == "2000" and ur_path:
        # Special case for 2000 blocks (of 2000 bgp)-- needs Urban/Rural code
        # For more details see:
        # https://gist.github.com/jGaboardi/36c7640af1f228cdc8a691505262e543
        ur_df = pandas.read_csv(ur_path, index_col=0, dtype=str)
        ur_df = ur_df[ur_df["STATE"] == stfips]
        blk_cols = ["STATE", "COUNTY", "TRACT", "BLOCK"]
        ur_df = nhgisxwalk.id_codes.blk_id(ur_df, blk_cols, )
        tab["URBRURALA"] = tab.index.map(dict(ur_df[["GISJOIN", "UR"]].values))
        # reorder columns
        cols = tab.columns
        reorder_cols = list(cols[:11]) + list(cols[-1:]) + list(cols[11:-1])
        tab = tab[reorder_cols]
        
    # 7. write out
    tab.to_csv("%s/%s.csv.zip"%(out_dir,f_tab))
    
    if aux_tabular:
        return xwalk, tab, aux
    else:
        return xwalk, tab

## Generate subset

In [4]:
subset_state = "09" # Connecticut
#subset_state = "10" # Delaware
#subset_state = "11" # DC
#subset_state = "56" # Wyoming

source_year, target_year = "1990", "2010"
#source_year, target_year = "2000", "2010"

In [5]:
# prep bgp1990_to_trt2010
if source_year == "1990" or target_year == "1990":
    xwalk_df, tab_df, aux_df = build_subset(
        subset_state,
        source_year,
        "blk",
        target_year,
        "blk",
        "block",
        tabular_dir="../../tabular_data",
        xwalk_dir="../../crosswalks",
        aux_tabular="blck_grp_598_103"
    )

# prep bgp2010_to_trt2010
if source_year == "2000" or target_year == "2000":
    xwalk_df, tab_df = build_subset(
        subset_state,
        source_year,
        "blk",
        target_year,
        "blk",
        "block",
        tabular_dir="../../tabular_data",
        xwalk_dir="../../crosswalks",
        ur_path="../../missing_UR_codes/2000_block_UR.csv.zip"
    )

/Users/jgaboardi/miniconda3/envs/nhgis_env/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [6]:
xwalk_df.head()
#xwalk_df[145:155]

,GJOIN1990,GJOIN2010,WEIGHT,PAREA_VIA_BLK00,st1990
0,NaN,G09000100108001015,0.0,0.0,NaN
1,NaN,G09000100108002010,0.0,0.0,NaN
2,NaN,G09000100110003010,0.0,0.0,NaN
3,NaN,G09000100110004005,0.0,0.0,NaN
4,NaN,G09000100110004019,0.0,0.0,NaN


In [7]:
xwalk_df.tail()

,GJOIN1990,GJOIN2010,WEIGHT,PAREA_VIA_BLK00,st1990
101079,G09001509081299,G09001509081002022,0.256335,0.256335,09
101080,G09001509081299,G09001509081002039,0.000000,0.000104,09
101081,G09001509081299,G09001509081002048,0.330787,0.330683,09
101082,G44000700133215,G09001509041003097,0.003269,0.000711,44
101083,G44000700133299,G09001509041003097,0.000000,0.000116,44


In [8]:
tab_df.head()

,YEAR,ANRCA,AIANHHA,RES_ONLYA,TRUSTA,RES_TRSTA,BLOCKA,BLCK_GRPA,TRACTA,CDA,...,STATE,STATEA,URBRURALA,URB_AREAA,CD103A,ANPSADPI,ET1001,EUD001,EUO001,ESA001
GISJOIN,,,,,,,,,,,,,,,,,,,,,
G0900010010101101,1990,99,9999,9999,9999,9,101,1,010101,04,...,Connecticut,09,1,8040,NaN,Block 101,67,0,1,1
G0900010010101102,1990,99,9999,9999,9999,9,102,1,010101,04,...,Connecticut,09,1,8040,NaN,Block 102,301,88,108,117
G0900010010101103,1990,99,9999,9999,9999,9,103,1,010101,04,...,Connecticut,09,1,8040,NaN,Block 103,18,4,6,7
G0900010010101104,1990,99,9999,9999,9999,9,104,1,010101,04,...,Connecticut,09,1,8040,NaN,Block 104,64,20,25,25
G0900010010101105,1990,99,9999,9999,9999,9,105,1,010101,04,...,Connecticut,09,1,8040,NaN,Block 105,166,47,51,53


In [9]:
tab_df.tail()

,YEAR,ANRCA,AIANHHA,RES_ONLYA,TRUSTA,RES_TRSTA,BLOCKA,BLCK_GRPA,TRACTA,CDA,...,STATE,STATEA,URBRURALA,URB_AREAA,CD103A,ANPSADPI,ET1001,EUD001,EUO001,ESA001
GISJOIN,,,,,,,,,,,,,,,,,,,,,
G09001509081247,1990,99,9999,9999,9999,9,247,2,9081,02,...,Connecticut,09,2,9999,NaN,Block 247,11,2,2,2
G09001509081249,1990,99,9999,9999,9999,9,249,2,9081,02,...,Connecticut,09,2,9999,NaN,Block 249,11,3,3,4
G09001509081251,1990,99,9999,9999,9999,9,251,2,9081,02,...,Connecticut,09,2,9999,NaN,Block 251,37,11,13,14
G09001509081253,1990,99,9999,9999,9999,9,253,2,9081,02,...,Connecticut,09,2,9999,NaN,Block 253,7,1,1,1
G09001509081257,1990,99,9999,9999,9999,9,257,2,9081,02,...,Connecticut,09,2,9999,NaN,Block 257,1,0,1,1


In [10]:
try:
    markdown_output = aux_df.head()
except NameError:
    markdown_output = "aux_df.head() does not exist."
markdown_output

,GISJOIN,YEAR,ANRCA,AIANHH,AIANHHA,RES_ONLYA,TRUSTA,RES_TRSTA,BLOCKA,BLCK_GRPA,...,STATEA,URBRURALA,URB_AREA,URB_AREAA,CD103A,ANPSADPI,ET1001,EUD001,EUO001,ESA001
0,G0900010047200479020019999999193011,1990,99,NaN,9999,9999,9999,9,NaN,1,...,09,1,"Danbury, CT--NY",1930,NaN,BG 1,1697,464,637,658
1,G0900010047200479020019999999193012,1990,99,NaN,9999,9999,9999,9,NaN,2,...,09,1,"Danbury, CT--NY",1930,NaN,BG 2 (pt.),1620,437,629,653
2,G0900010047200479020029999999193011,1990,99,NaN,9999,9999,9999,9,NaN,1,...,09,1,"Danbury, CT--NY",1930,NaN,BG 1,1876,473,806,846
3,G0900010047200479020029999999193012,1990,99,NaN,9999,9999,9999,9,NaN,2,...,09,1,"Danbury, CT--NY",1930,NaN,BG 2,823,207,374,391
4,G0900010047200479020029999999193013,1990,99,NaN,9999,9999,9999,9,NaN,3,...,09,1,"Danbury, CT--NY",1930,NaN,BG 3,883,236,318,325


In [11]:
try:
    markdown_output = aux_df.tail()
except NameError:
    markdown_output = "aux_df.tail() does not exist."
markdown_output

,GISJOIN,YEAR,ANRCA,AIANHH,AIANHHA,RES_ONLYA,TRUSTA,RES_TRSTA,BLOCKA,BLCK_GRPA,...,STATEA,URBRURALA,URB_AREA,URB_AREAA,CD103A,ANPSADPI,ET1001,EUD001,EUO001,ESA001
4101,G4400070807808078001849999999648014,1990,99,NaN,9999,9999,9999,9,NaN,4,...,44,1,"Providence--Pawtucket, RI--MA",6480,NaN,BG 4,825,242,297,327
4102,G4400070807808078001849999999648015,1990,99,NaN,9999,9999,9999,9,NaN,5,...,44,1,"Providence--Pawtucket, RI--MA",6480,NaN,BG 5,2108,530,708,734
4103,G4400070807808078001859999999648011,1990,99,NaN,9999,9999,9999,9,NaN,1,...,44,1,"Providence--Pawtucket, RI--MA",6480,NaN,BG 1,796,222,321,335
4104,G4400070807808078001859999999648012,1990,99,NaN,9999,9999,9999,9,NaN,2,...,44,1,"Providence--Pawtucket, RI--MA",6480,NaN,BG 2,1244,354,487,508
4105,G4400070807808078001859999999648013,1990,99,NaN,9999,9999,9999,9,NaN,3,...,44,1,"Providence--Pawtucket, RI--MA",6480,NaN,BG 3,928,268,325,329


----------------------------
## Test crosswalk build with generated subset

In [ ]:
subset_data_dir = "../testing_data_subsets"
base_xwalk_name = "/nhgis_blk%s_blk%s_gj.csv.zip" % (source_year, target_year)
base_xwalk_file = subset_data_dir + base_xwalk_name
data_types = nhgisxwalk.str_types(["GJOIN%s"%source_year, "GJOIN%s"%target_year])
base_xwalk = pandas.read_csv(base_xwalk_file, index_col=0, dtype=data_types)
base_xwalk.head()

In [ ]:
invc = nhgisxwalk.desc_code_1990
#invc = nhgisxwalk.desc_code_2000_SF1b

input_var_names = ["Persons", "Families", "Households", "Housing Units"]
input_var_codes = [invc[name]["Total"] for name in input_var_names]
input_var_codes

In [ ]:
input_var_tags = ["pop", "fam", "hh", "hu"]

In [ ]:
'''
bgp1990_to_trt2010 = nhgisxwalk.GeoCrossWalk(
    base_xwalk,
    source_year=source_year,
    target_year=target_year,
    source_geo="bgp",
    target_geo="trt",
    base_source_table=subset_data_dir+"/1990_block.csv.zip",
    input_var=input_var_codes,
    weight_var=input_var_tags,
    stfips=subset_state
)

'''
#'''
bgp2000_to_trt2010 = nhgisxwalk.GeoCrossWalk(
    base_xwalk,
    source_year=source_year,
    target_year=target_year,
    source_geo="bgp",
    target_geo="trt",
    base_source_table=subset_data_dir+"/2000_block.csv.zip",
    input_var=input_var_codes,
    weight_var=input_var_tags,
    stfips=subset_state
)
#'''
#bgp1990_to_trt2010.xwalk.head()
bgp2000_to_trt2010.xwalk.head()

-----------------------

## Data for unittests and doctests
### Delaware
**base_xwalk_blk1990_blk2010**

In [ ]:
wgt_cols = ["wt_pop", "wt_fam", "wt_hh", "wt_hu"]
id_cols = ["bgp2000", "trt2010"]

In [ ]:
#ix1, ix2 = 677, 681

#ix1, ix2 = 1025, 1029 # bgp1990_to_trt2010
#bgp1990_to_trt2010.xwalk[ix1:ix2]

**base_xwalk_blk2000_blk2010**

In [ ]:
# doctest
print(bgp2000_to_trt2010.xwalk[id_cols+wgt_cols[:2]][1020:1031])

In [ ]:
bgp2000_to_trt2010.xwalk.head()

In [ ]:
ix1, ix2 = 1025, 1029 # bgp2000_to_trt2010
bgp2000_to_trt2010.xwalk[ix1:ix2]

In [ ]:
bgp2000_to_trt2010.xwalk[id_cols][ix1:ix2].values

In [ ]:
knw_str_vals = numpy.array(
    [
        ["G10000509355299999051304R1", "G1000050051305"],
        ["G10000509355299999051304R1", "G1000050051306"],
        ["G10000509355299999051304R1", "G1000050051400"],
        ["G10000509355299999051304R2", "G1000050051305"]
    ]
)
knw_str_vals

In [ ]:
numpy.equal(
    bgp2000_to_trt2010.xwalk[id_cols][ix1:ix2].values,
    knw_str_vals
).all()

In [ ]:
bgp2000_to_trt2010.xwalk[wgt_cols][ix1:ix2].values

In [ ]:
knw_num_vals = numpy.array(
    [
        [6.80605382e-01, 6.33909150e-01, 6.57366450e-01, 6.59501671e-01],
        [3.19167389e-01, 3.65781711e-01, 3.42281879e-01, 3.40110906e-01],
        [2.27229039e-04, 3.09138740e-04, 3.51671251e-04, 3.87423412e-04],
        [8.02660754e-01, 8.17567568e-01, 8.20895522e-01, 8.36236934e-01]
    ]
)
knw_num_vals

In [ ]:
numpy.allclose(
    bgp2000_to_trt2010.xwalk[wgt_cols][ix1:ix2].values,
    knw_num_vals
)

-------------------------

### DC
**base_xwalk_blk1990_blk2010**

In [ ]:
#ix1, ix2 = 688, 692

#knw_str_vals = numpy.array(
#    [
#        ["G11000105000050000009806989999999884011", "G1100010009811"],
#        ["G11000105000050000009806989999999884012", "G1100010009810"],
#        ["G11000105000050000009806989999999884012", "G1100010009811"],
#        ["G11000105000050000009807989999999884011", "G1100010009807"],
#    ]
#)
#knw_num_vals = numpy.array(
#    [
#        [1.0, 1.0, 1.0, 1.0],
#        [0.41477113, 0.41545353, 0.39687267, 0.39506995],
#        [0.58522887, 0.58454647, 0.60312733, 0.60493005],
#        [1.0, 1.0, 1.0, 1.0],
#    ]
#)
#obs_xwalk = nhgisxwalk.GeoCrossWalk(
#    base_xwalk_blk1990_blk2010,
#    source_year=_90,
#    target_year=_10,
#    source_geo=bgp,
#    target_geo=trt,
#    base_source_table=tab_data_path_1990,
#    input_var=input_vars_1990,
#    weight_var=input_var_tags,
#)
#ix1, ix2 = 688, 692
#id_cols = ["bgp1990", "trt2010"]
#obs_str_vals = obs_xwalk.xwalk[id_cols][ix1:ix2].values
#wgt_cols = ["wt_pop", "wt_fam", "wt_hh", "wt_hu"]
#obs_num_vals = obs_xwalk.xwalk[wgt_cols][ix1:ix2].values
#numpy.testing.assert_equal(knw_str_vals, obs_str_vals)
#numpy.testing.assert_allclose(knw_num_vals, obs_num_vals)

**base_xwalk_blk2000_blk2010**

In [ ]:
#ix1, ix2 = 677, 681

#knw_str_vals = numpy.array(
#    [
#        ["G1101050000500009806R2", "G1100010009810"],
#        ["G1101050000500009806U1", "G1100010009811"],
#        ["G1101050000500009806U2", "G1100010009810"],
#        ["G1101050000500009806U2", "G1100010009811"],
#    ]
#)
#knw_num_vals = numpy.array(
#    [
#        [0.0, 0.0, 0.0, 0.0],
#        [1.0, 1.0, 1.0, 1.0],
#        [0.4234478601567, 0.4310747663551, 0.404344193817, 0.4043715846994],
#        [0.5765521398432, 0.5689252336448, 0.595655806182, 0.5956284153005],
#    ]
#)
#obs_xwalk = nhgisxwalk.GeoCrossWalk(
#    base_xwalk_blk2000_blk2010,
#    source_year=_00,
#    target_year=_10,
#    source_geo=bgp,
#    target_geo=trt,
#    base_source_table=tab_data_path_2000,
#    input_var=input_vars_2000_SF1b,
#    weight_var=input_var_tags,
#)
#ix1, ix2 = 677, 681
#id_cols = ["bgp2000", "trt2010"]
#obs_str_vals = obs_xwalk.xwalk[id_cols][ix1:ix2].values
#wgt_cols = ["wt_pop", "wt_fam", "wt_hh", "wt_hu"]
#obs_num_vals = obs_xwalk.xwalk[wgt_cols][ix1:ix2].values
#numpy.testing.assert_equal(knw_str_vals, obs_str_vals)
#numpy.testing.assert_allclose(knw_num_vals, obs_num_vals)

### Wyoming
**base_xwalk_blk1990_blk2010**

**base_xwalk_blk2000_blk2010**

-----------------